# Accessing and Processing Data from APIs (REST and SOAP) in Python




## 1. REST APIs

REST APIs are accessed via HTTP requests. We'll cover GET (retrieve data), POST (create data), and processing responses.

Representational State Transfer APIs are stateless and use HTTP methods like GET, POST, PUT, DELETE. They often return data in JSON format.

We'll use the `requests` library for REST APIs

### Key Concepts:
- Use `requests.get()`, `requests.post()`, etc.
- Handle JSON responses with `response.json()`.
- Error handling with status codes.
- Authentication (e.g., API keys).

In [3]:
# pip install requests  #install requests library

In [4]:
import requests
import pandas as pd

### Example 1: Simple GET Request (JSONPlaceholder)

Fetch todo items from a fake REST API.
JSONPlaceholder is a free fake REST API created for:

- Learning REST APIs
- Practicing frontend/backend development
- Testing HTTP methods (GET, POST, PUT, DELETE)
- It does NOT save data permanently. All write operations are simulated.

In [5]:
# GET request
url = 'https://jsonplaceholder.typicode.com/todos'
params = {'_limit': 10}# Query parameters
response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    print('Fetched data:')
    print(data)
else:
    print(f'Error: {response.status_code}')

Fetched data:
[{'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False}, {'userId': 1, 'id': 2, 'title': 'quis ut nam facilis et officia qui', 'completed': False}, {'userId': 1, 'id': 3, 'title': 'fugiat veniam minus', 'completed': False}, {'userId': 1, 'id': 4, 'title': 'et porro tempora', 'completed': True}, {'userId': 1, 'id': 5, 'title': 'laboriosam mollitia et enim quasi adipisci quia provident illum', 'completed': False}, {'userId': 1, 'id': 6, 'title': 'qui ullam ratione quibusdam voluptatem quia omnis', 'completed': False}, {'userId': 1, 'id': 7, 'title': 'illo expedita consequatur quia in', 'completed': False}, {'userId': 1, 'id': 8, 'title': 'quo adipisci enim quam ut ab', 'completed': True}, {'userId': 1, 'id': 9, 'title': 'molestiae perspiciatis ipsa', 'completed': False}, {'userId': 1, 'id': 10, 'title': 'illo est ratione doloremque quia maiores aut', 'completed': True}]


### Processing REST Data

Convert JSON to a pandas DataFrame for analysis.

In [6]:
# Process into DataFrame
df_todos = pd.DataFrame(data)
print('Todos DataFrame:')
print(df_todos.head())

# Example analysis: Count completed todos
completed_count = df_todos['completed'].sum()
print(f'Completed todos: {completed_count}')

Todos DataFrame:
   userId  id                                              title  completed
0       1   1                                 delectus aut autem      False
1       1   2                 quis ut nam facilis et officia qui      False
2       1   3                                fugiat veniam minus      False
3       1   4                                   et porro tempora       True
4       1   5  laboriosam mollitia et enim quasi adipisci qui...      False
Completed todos: 3


In [5]:
# Convert to csv

df_todos.to_csv('todos.csv', index=False)
print('Data saved to todos.csv')

Data saved to todos.csv


### Example 2: POST Request (Create Data)

Send data to create a new post.


In [7]:
# POST request
post_url = 'https://jsonplaceholder.typicode.com/posts'
payload = {
    'title': 'Hello World',
    'body': 'This is a test post.',
    'userId': 1
}
headers = {'Content-Type': 'application/json'}

post_response = requests.post(post_url, json=payload, headers=headers)

if post_response.status_code == 201:
    created_data = post_response.json()
    print('Created post:')
    print(created_data)
else:
    print(f'Error: {post_response.status_code}')

Created post:
{'title': 'Hello World', 'body': 'This is a test post.', 'userId': 1, 'id': 101}


### Example 3: Authenticated GET (OpenWeatherMap)

Fetch weather data. Replace 'YOUR_API_KEY' with a real key from https://openweathermap.org/api.

In [ ]:
# Authenticated GET with API key
weather_url = 'https://api.openweathermap.org/data/2.5/weather'
params = {
    'q': 'Pokhara',
    # 'appid': ''  # Replace with your actual API key
}

weather_response = requests.get(weather_url, params=params)

if weather_response.status_code == 200:
    weather_data = weather_response.json()
    print('Weather data:')
    print(weather_data)
else:
    print(f'Error: {weather_response.status_code} - {weather_response.text}')

Weather data:
{'coord': {'lon': 83.9833, 'lat': 28.2333}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 293.23, 'feels_like': 292.02, 'temp_min': 293.23, 'temp_max': 293.23, 'pressure': 1014, 'humidity': 28, 'sea_level': 1014, 'grnd_level': 872}, 'visibility': 10000, 'wind': {'speed': 3.25, 'deg': 139, 'gust': 2.12}, 'clouds': {'all': 98}, 'dt': 1769336201, 'sys': {'country': 'NP', 'sunrise': 1769303688, 'sunset': 1769342249}, 'timezone': 20700, 'id': 1282898, 'name': 'Pokhara', 'cod': 200}


### Processing Weather Data

Extract and display key information.

In [12]:
if weather_response.status_code == 200:
    temp = weather_data['main']['temp'] - 273.15  # Convert Kelvin to Celsius
    description = weather_data['weather'][0]['description']
    print(f'Temperature in Pokhara: {temp:.2f}°C')
    print(f'Weather: {description}')

Temperature in Pokhara: 20.08°C
Weather: overcast clouds


## 2. SOAP APIs

Simple Object Access Protocol APIs are protocol-based, using XML for messaging. They are more structured and often used in enterprise environments.

SOAP APIs use WSDL (Web Services Description Language) to define services. We'll use `zeep` to create a client.

SOAP is a heavy, XML-based protocol with strict rules, while REST is a lightweight, flexible architectural style that typically uses JSON and HTTP methods.

We'll use the `zeep` library  for SOAP.

### Key Concepts:
- Load WSDL to create a client.
- Call service methods.
- Handle XML responses.
- Convert to Python structures.

### Example: Public SOAP Service (CountryInfo)

Fetch country information using http://webservices.oorsprong.org/websamples.countryinfo/CountryInfoService.wso.

In [14]:
# install zeep library for SOAP API
# pip install zeep

In [15]:
from zeep import Client

In [16]:
# Create SOAP client
wsdl = 'http://webservices.oorsprong.org/websamples.countryinfo/CountryInfoService.wso?WSDL'
client = Client(wsdl)

# Call a method: List of currencies
currencies = client.service.ListOfCurrenciesByName()
print('Currencies response type:', type(currencies))
print('First few currencies:')
for item in currencies[:5]:
    print(item)

Currencies response type: <class 'list'>
First few currencies:
{
    'sISOCode': 'AFA',
    'sName': 'Afghanis'
}
{
    'sISOCode': 'DZD',
    'sName': 'Algeria Dinars'
}
{
    'sISOCode': 'MGA',
    'sName': 'Ariary'
}
{
    'sISOCode': 'AWG',
    'sName': 'Aruba Guilders (also called Florins)'
}
{
    'sISOCode': 'AUD',
    'sName': 'Australian Dollars'
}


### Processing SOAP Data

Convert the response to a list of dictionaries and then to a DataFrame.

In [22]:
# Extract data
currency_list = []
for currency in currencies:
    currency_list.append({
        'ISOCode': currency['sISOCode'],
        'Name': currency['sName']
    })

# To DataFrame
df_currencies = pd.DataFrame(currency_list)
print('Currencies DataFrame:')
print(df_currencies.head(10))

# Example: Filter by name
usd = df_currencies[df_currencies['Name'].str.contains('Dollar', na=False)]
print('\nDollar currencies:')
print(usd)

Currencies DataFrame:
  ISOCode                                  Name
0     AFA                              Afghanis
1     DZD                        Algeria Dinars
2     MGA                                Ariary
3     AWG  Aruba Guilders (also called Florins)
4     AUD                    Australian Dollars
5     BSD                       Bahamas Dollars
6     BHD                        Bahrain Dinars
7     THB                                  Baht
8     PAB                                Balboa
9     BBD                      Barbados Dollars

Dollar currencies:
    ISOCode                    Name
4       AUD      Australian Dollars
5       BSD         Bahamas Dollars
9       BBD        Barbados Dollars
10      BZD          Belize Dollars
11      BMD         Bermuda Dollars
16      BND          Brunei Dollars
17      CAD        Canadian Dollars
18      KYD   Cayman Island Dollars
39      FJD                 Dollars
40      GYD                 Dollars
41      JMD                 Dollar

### Another SOAP Method: Country Capital

Fetch the capital of a country.

In [26]:
# Call method with parameter
capital = client.service.CapitalCity(sCountryISOCode='NP')  # Nepal
print(f'Capital of Nepal: {capital}')

Capital of Nepal: Katmandu


## Best Practices
- Handle exceptions (e.g., network errors) using try-except.
- For production, use environment variables for API keys.
- Rate limiting: Respect API limits to avoid bans.
- For large data, use streaming or pagination.
- Security: Use HTTPS and validate inputs.